<font color='blue'> Привет!) Поздравляю с первым самостоятельным проектом. Ты проделал большую работу. Далее в файле ты можешь найти мои комменатрии, выделенные синим. Пожайлуста, постарайся учесть их в дальнейших проектах.</font>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Глядя на исходную таблицу, можно сделать выводы: 
    1. Часть данных представлена в не корректном виде, в частности стаж работы представлен как отрицательные числа, а так же длинная дробная часть;
    2. Есть некорректные данные в days_employed, а именно сроки по 100-900 лет (в пересчете на года);
    3. Образование указано в разных регистрах.
    4. Название столбцов - корректные;
    5. Типы данных в столбцах в целом корректны, частично надо поменять тип с вещественных на целочисленные.
Это первый визуальный анализ, дальше мы разберем исходные данные глубже.

<font color='blue'> Хороший анализ сырых данных.</font>

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#определяем количество пропусков
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

По данным выше видно, что пропуски у нас имеются в двух столбцах (опыт работы в днях, ежемесячный доход), причина появления пропусков - человек не указал свой доход и стаж работы. 
Просто удалить пропуски из таблицы было бы не корректно, это примерно 10% данных, при условии 100% пересечения пропусков в двух столбцах, потенциально 20%. 
Чтобы не потерять эти данный, их надо заполнить. 
В случае со столбцом total_income проблем нет, подставим в пропуски медиану.
В случае со столбцом days_employed проблемы есть, внутри много отрицательных чисел (необходимо взять столбец по модулю), а так же внутри имеются данные, где стаж в годах варьируется от 100 до 900 лет!!! Эти значения очень плохо скажутся на общем усреднении, их надо скорректировать.
Основная идея корректировки - если трудовой стаж больше возраста человека то заменяем его на 65% возраста человека (65% - подобрано эмпирическим путем).

In [3]:
#Поменяем значения во всех числовых столбцах на положительные (берем модуль числа)
data['days_employed'] = abs(data['days_employed'])
data['total_income'] = abs(data['total_income'])
data['children'] = abs(data['children'])
data['dob_years'] = abs(data['dob_years'])
data['education_id'] = abs(data['education_id'])
data['family_status_id'] = abs(data['family_status_id'])
data['debt'] = abs(data['debt'])

#Если стаж больше возраста, то стаж равен 65% возраста человека
data.loc[data['days_employed']>=data['dob_years']*365, 'days_employed'] = data['dob_years']*365*0.65 

#Заполним пропуски в days_employed медианным значением столбца
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

#Заполним пропуски в total_income медианным значением столбца
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

#Проверим наличие пропусков теперь
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Мы избавились от пропусков в наших исходных данных, не потеряв ни одной строки.

<font color='blue'> Все пропуски заполнили отлично.</font>

### Замена типа данных

In [4]:
#В исходных данных столбцы days_employed и total_income представленны в вещественном виде. 
#Нам такая точность не нужна ни по одному из столбцов, переведем их в целочисленный тип.
#Применим метод astype
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
#Проверим типы данных столбцов с помощью info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Заменили типы данных методом astype, который округляет до целого, знаки после запятой в вещественных числах на текущих данных - избыточная точность, никак не влияющая на результат исследования.

<font color='blue'> Тип данных приведен к норме - это важный шаг.</font>

### Поиск дубликатов и выпадающих значений, удаление "лишних"

In [5]:
#Избавимся от дубликатов с учетом регистра в столбце education
data['education'] = data['education'].str.lower()

#В файле есть 76 строк, где заявляется, что имеют 20 детей, предполагаю, что это ошибочно допечатали 0 
#Уберем эту ошибку из данных.
data.loc[data['children']==20, 'children'] = 2

#В данных есть строки, где значение dob_years (возраст) равны 0, 101 строка
#Удалим их
data = data.loc[data['dob_years'] != 0]

#В данных есть строка, где значение gender (пол) равны XNA, удаляем
data = data.loc[data['gender'] != 'XNA']

#В данных есть несколько типов занятости, которые выпадают из общих (6 строк) - удаляем.
data = data.loc[data['income_type'] != 'предприниматель']
data = data.loc[data['income_type'] != 'безработный']
data = data.loc[data['income_type'] != 'студент']
data = data.loc[data['income_type'] != 'в декрете']

#Проверим наличие дубликатов
data.duplicated().sum() 
#Находим 71 дубликат

#Удаляем дубликаты
data.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,12574,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21341,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21342,0,15895,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21343,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21344,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

В исходных данных нашолся 71 дубликат, которые могли там оказаться, по причине оформления нескольких заявок на кредит одними и теми же лицами (соответственно указывали идентичные данные).
Так же удалили из таблицы заведомо ошибочные данные, чтобы подготовить таблицу к полноценному анализу без "мусора".

<font color='blue'>Здесь лучше напрямую выводить, а не писать в комментариях значения. Для того, чтоб вывелось на экран количество дубликатов надо либо использовать print(), либо писать в отдельной ячейке (по умолчанию на экран выводится значение последней строки ячейки). В конце лучше было бы еще раз вывести количество дубликатов - их должно стать 0, так мы покажем, что все дубли действительно удалились.</font>

### Лемматизация

In [6]:
#Подключим библиотеку pymystem3, для работы с лемматизацией
from pymystem3 import Mystem
m = Mystem()

#Функция категорирования, на вход отдаем строку, лемматизируем ее,
#если хоть одно слово из лемматизированной строки совпадает с категорией, то выдаем категорию
def categorizer(string):
    category_list = ['свадьба', 'автомобиль', 'образование','недвижимость', 'жилье']
    lemma = []
    lemma = m.lemmatize(string)
    for row in range(len(category_list)):
        for i in range(len(lemma)):
            if lemma[i-1] == category_list[row-1]:
                if category_list[row-1]=='жилье':
                    return 'недвижимость'
                return category_list[row-1]
            
#Добавим новый столбец с категоризированными данными
data['credit_target'] = data['purpose'].apply(categorizer)

### Вывод

При просмотре исходных данных, выделил явные четыре категории: недвижимость, автомобиль, образование и свальда.
Катотегоризировали кредитные запросы от клиентов лемматизировав их запросы и сравнив с категориями. 
Категории запросов добавили в отдельный столбец, анализ проведем ниже.

<font color='blue'> Отлично, лемматизация сделана- это самая сложная часть проекта, а заодно и категоризацию захватил. Только вконце надо было вывести head или value_counts, чтоб посмотреть результат.</font>

### Категоризация данных

In [7]:
#Категоризириуем уровень дохода следующей градацией:
#0-50000 - "очень низкий"
#50001 - 100000 - "низкий"
#100001 - 150000 - "средний"
#151000 - 250000 - "высокий"
#250000 и выше - "очень высокий"
#Создадим функцию для категоризации
def income_group(messages):
    if 0 < messages <= 50000:
        return 'очень низкий'
    if 50001 <= messages <= 100000:
        return 'низкий'
    if 100001 <= messages <= 150000:
        return 'средний'
    if 150001 <= messages <= 250000:
        return 'высокий'
    return 'очень высокий'

#Напишем функцию, которая категорезирует по признаку есть/нет дети
def children_group(messages):
    if messages > 0:
        return 'Есть дети'
    return 'Нет детей'

#Добавим новый столбец с категоризацией по уровню ежемесячного дохода
data['income_group'] = data['total_income'].apply(income_group)

#Добавим новый столбец с категоризацией по наличию детей
data['children_group'] = data['children'].apply(children_group)

<font color='blue'> Для категоризации по уровню дохода лучше использовать квантили. Категоризировать количество детей не следует - это снижает информативность данных.</font>

### Вывод

Категоризировали клиентов по уровню дохода, критерии были выбраны из существующих реалий и ощущений:
0-50 - очень низкие
50-100 - низкие
100-150 - средние
150-250 - высокие
250+ - очень высокие.
Дальнейший анализ распределения просрочек проведем ниже.

Так же провели категоризацию данных по наличию/отсутствию детей.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
#Агрегируем новую таблицу из имеющихся данных, сгруппируем по наличию детей, добавим сколько всего таких статусов
#и сколько в каждом статусе просрочек
#Вычислим процент просрочки в каждом статусе
children_grouped = data.groupby('children_group').agg({'debt': ['count', 'sum']})
children_grouped['percent_debt'] = children_grouped['debt']['sum'] / children_grouped['debt']['count']*100
children_grouped.sort_values(by='percent_debt', ascending=False)

debt       percent_debt
                count   sum             
children_group                          
Есть дети        7342   673     9.166440
Нет детей       14075  1058     7.516874

### Вывод

По полученным данным можно сделать вывод, что зависимость от наличия/отсутствия детей имеется, 9.17% клиентов, у которых есть дети имели просроченную задолженность, по сравнению с 7.52% клиентов без детей.
Клиенты с детьми будут иметь более рискованный кейс, чем клиенты без детей.

<font color='blue'> Если не категорехзоровать, то можно вывести более тонкие зависимости не только от наличия, но и от количества детей.</font>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
#Агрегируем новую таблицу из имеющихся данных, сгруппируем по семейному статусу, добавим сколько всего таких статусов
#и сколько в каждом статусе просрочек
#Вычислим процент просрочки в каждом статусе
family_grouped = data.groupby('family_status').agg({'debt': ['count', 'sum']})
family_grouped['percent_debt'] = family_grouped['debt']['sum'] / family_grouped['debt']['count']*100
family_grouped.sort_values(by='percent_debt', ascending=False)

debt      percent_debt
                       count  sum             
family_status                                 
Не женат / не замужем   2796  273     9.763948
гражданский брак        4153  386     9.294486
женат / замужем        12328  925     7.503245
в разводе               1185   85     7.172996
вдовец / вдова           955   62     6.492147

### Вывод

По полученным результатам можно сделать вывод, что клиенты, у которых нет и не было законного брака имели больше просрочек по кредитам примерно на 2%, чем те, кто был в браке и те, кто развелся. 
Меньше всего просроченной задолжности было у вдовцов/вдов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
income_grouped = data.groupby('income_group').agg({'debt': ['count', 'sum']})
income_grouped['percent_debt'] = income_grouped['debt']['sum'] / income_grouped['debt']['count']*100
income_grouped.sort_values(by='percent_debt', ascending=False)

debt      percent_debt
              count  sum             
income_group                         
средний        7845  658     8.387508
высокий        6337  529     8.347799
низкий         4067  328     8.064913
очень высокий  2798  193     6.897784
очень низкий    370   23     6.216216

### Вывод

По полученным результатам можно сделать вывод, что клиенты, с низким/средним/высоким (с доходом от 50 до 250) достатком имели примерно одинаковые проценты просроченной задолженности. В то время как люди с очень высокими и очень низкими доходами имели меньшую просроченную задолженность по кредитам.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
target_grouped = data.groupby('credit_target').agg({'debt': ['count', 'sum']})
target_grouped['percent_debt'] = target_grouped['debt']['sum'] / target_grouped['debt']['count']*100
target_grouped.sort_values(by='percent_debt', ascending=False)

debt      percent_debt
               count  sum             
credit_target                         
автомобиль      4292  399     9.296365
образование     4004  370     9.240759
свадьба         2333  184     7.886841
недвижимость   10788  778     7.211717

### Вывод

По полученным данным можно сделать вывод, что цель кредита ьак же влияет на рискованность кредитного портфеля, а именно, можно увидеть что процент просрочек был выше при кредитах на автомобили и образование.
Менее рискованными можем считать кредиты на свадьбу. По кредитам на недвижимость просрочек бывает меньше всего.

<font color='blue'> Молодец, ответы на вопрос ввыполнены хорошо.</font>

### Шаг 4. Общий вывод

По результату данного исследования, можно сделать вывод, что на вероятноть возникновения просроченной задолжности у клиента влияет множество факторов и в то же время, разброс количества просроченной задолженности в кредитном кейсе составляет от 6.5% до 9.5% по всем исследуемым парамерам. 
Если мы усредним данный разброс, то получим примерно 8% - размер предполагаемого сумарного количества клиентов с просроченной задолженностью, который можно закладывать при формировании бизнес-модели.

<font color='blue'> Проект выполнен на очень хорошем уровне. Но общие выводы можно расписать поподробнее, часто заказчик читает именно итоговые или общие выводы. Удачи в следующих проектах</font>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.